In [1]:
import csv
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
fmt = "%Y-%m-%d %X"
datetime_delta = timedelta(minutes=20)
train_cases = {}
train_days = []
with open('training2_20min_avg_volume.csv') as f:
    f.readline()
    reader = csv.reader(f)
    for row in reader:
        start_time = row[1].split(',')[0][1:]
        day = start_time.split()[0]
        if day not in train_days:
            train_days.append(day)
        index = row[0]+'-'+row[2]+'-'+start_time
        train_cases[index]=float(row[3])

cases = {}
days=[]
with open('test1_20min_avg_volume.csv') as f:
    f.readline()
    reader = csv.reader(f)
    for row in reader:
        start_time = row[1].split(',')[0][1:]
        day = start_time.split()[0]
        if day not in days:
            days.append(day)
        index = row[0]+'-'+row[2]+'-'+start_time
        cases[index]=float(row[3])

In [ ]:
true_result = []
for id in range(1,4):
    for dir in range(0,2):
        if id == 2 and dir ==1:
            break;
        for day in days:
            periods = np.zeros([72])
            curr_datetime = datetime.strptime(day+' 00:20:00',fmt)
            for i in range(72):
                index = str(id)+'-'+str(dir)+'-'+curr_datetime.strftime(fmt)
                curr_datetime += datetime_delta
                if index in train_cases:
                    periods[i] = train_cases[index]
            for i in range(23,29):
                true_result.append(periods[i])
            for i in range(50,56):
                true_result.append(periods[i])

In [ ]:
m = 35.0
step = 0.05
best_mape = 1.0
best_m = m
while m<40.0:
    output = open('submission.csv','w')
    output.write('tollgate_id,time_window,direction,volume\n')
    predict_result = []
    for id in range(1,4):
        for dir in range(0,2):
            if id == 2 and dir ==1:
                break
            for day in days:
                periods = np.zeros([72])
                curr_datetime = datetime.strptime(day+' 00:20:00',fmt)
                for i in range(72):
                    index = str(id)+'-'+str(dir)+'-'+curr_datetime.strftime(fmt)
                    curr_datetime += datetime_delta
                    if index in cases:
                        periods[i] = cases[index]
                x_train = np.array([[i,periods[i]] for i in range(17,23)])
                x_train[:,0]=x_train[:,0]-m
                x_train[:,0]=np.square(x_train[:,0])
                a,b=np.polyfit(x_train[:,0],x_train[:,1],1)
                x_test = np.array([23,24,25,26,27,28])
                y_test = a*np.square(x_test-m)+b
                for i in range(6):
                    begin_datetime = datetime.strptime(day+' 08:00:00',fmt)+i*datetime_delta
                    end_datetime = begin_datetime+datetime_delta
                    result = "%d,\"[%s,%s)\",%d,%f\n" %(id,begin_datetime.strftime(fmt),end_datetime.strftime(fmt),dir,y_test[i])
                    output.write(result)
                    predict_result.append(y_test[i])

                x_train = np.array([[i,periods[i]] for i in range(17,23)]+[[i,periods[i]] for i in range(44,50)])
                x_train[:,0]=x_train[:,0]-m
                x_train[:,0]=np.square(x_train[:,0])
                a,b=np.polyfit(x_train[:,0],x_train[:,1],1)
                x_test = np.array([50,51,52,53,54,55])
                y_test = a*np.square(x_test-m)+b
                for i in range(6):
                    begin_datetime = datetime.strptime(day+' 17:00:00',fmt)+i*datetime_delta
                    end_datetime = begin_datetime+datetime_delta
                    result = "%d,\"[%s,%s)\",%d,%f\n" %(id,begin_datetime.strftime(fmt),end_datetime.strftime(fmt),dir,y_test[i])
                    output.write(result)
                    predict_result.append(y_test[i])
    output.close()
    predict_result = np.array(predict_result)
    true_result = np.array(true_result)
    MAPE = np.average(np.abs(true_result-predict_result)/true_result)
    if MAPE < best_mape:
        best_mape = MAPE
        best_m = m
    m += step

In [ ]:
print(best_mape,best_m)

In [2]:
m = 36
output = open('phase1_submission.csv','w')
output.write('tollgate_id,time_window,direction,volume\n')
predict_result = []
for id in range(1,4):
    for dir in range(0,2):
        if id == 2 and dir ==1:
            break
        for day in days:
            periods = np.zeros([72])
            curr_datetime = datetime.strptime(day+' 00:20:00',fmt)
            for i in range(72):
                index = str(id)+'-'+str(dir)+'-'+curr_datetime.strftime(fmt)
                curr_datetime += datetime_delta
                if index in cases:
                    periods[i] = cases[index]
            x_train = np.array([[i,periods[i]] for i in range(17,23)])
            x_train[:,0]=x_train[:,0]-m
            x_train[:,0]=np.square(x_train[:,0])
            a,b=np.polyfit(x_train[:,0],x_train[:,1],1)
            x_test = np.array([23,24,25,26,27,28])
            y_test = a*np.square(x_test-m)+b
            for i in range(6):
                begin_datetime = datetime.strptime(day+' 08:00:00',fmt)+i*datetime_delta
                end_datetime = begin_datetime+datetime_delta
                result = "%d,\"[%s,%s)\",%d,%f\n" %(id,begin_datetime.strftime(fmt),end_datetime.strftime(fmt),dir,y_test[i])
                output.write(result)
                predict_result.append(y_test[i])

            x_train = np.array([[i,periods[i]] for i in range(17,23)]+[[i,periods[i]] for i in range(44,50)])
            x_train[:,0]=x_train[:,0]-m
            x_train[:,0]=np.square(x_train[:,0])
            a,b=np.polyfit(x_train[:,0],x_train[:,1],1)
            x_test = np.array([50,51,52,53,54,55])
            y_test = a*np.square(x_test-m)+b
            for i in range(6):
                begin_datetime = datetime.strptime(day+' 17:00:00',fmt)+i*datetime_delta
                end_datetime = begin_datetime+datetime_delta
                result = "%d,\"[%s,%s)\",%d,%f\n" %(id,begin_datetime.strftime(fmt),end_datetime.strftime(fmt),dir,y_test[i])
                output.write(result)
                predict_result.append(y_test[i])
output.close()